In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering,AutoModel,AutoConfig
from transformers import AutoTokenizer,AutoModelForQuestionAnswering, TrainingArguments, Trainer,AutoConfig,AutoModel
from transformers import DefaultDataCollator
from transformers import TrainingArguments
from transformers import HfArgumentParser
from transformers import Trainer
from datasets import load_dataset
import torch
from transformers import DistilBertModel
from datasets import load_dataset
from transformers import PreTrainedModel,PretrainedConfig
from transformers.modeling_outputs import QuestionAnsweringModelOutput
import numpy as np
import re
import string
import collections
from datasets import DatasetDict
import ast
from transformers import EarlyStoppingCallback
from tqdm import tqdm
import pandas as pd

In [7]:
para = "Tennis is a racket sport that is played either individually against a single opponent (singles) or between two teams of two players each (doubles). Each player uses a tennis racket that is strung with cord to strike a hollow rubber ball covered with felt over or around a net and into the opponent's court. The object of the game is to manoeuvre the ball in such a way that the opponent is not able to play a valid return. The player who is unable to return the ball validly will not gain a point, while the opposite player will."
question = "What is the ball made of?"

In [2]:

# Metric calculation taken from https://rajpurkar.github.io/SQuAD-explorer/
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

# Load through Pipline

In [5]:
# pipeline("question-answering", model="/home/vp.shivasan/interiit/task2/training_dir/task2_MiniLM-mod_50epochs_2e-5_FINAL_ES/checkpoint-5076")

# Load from base model

In [3]:
# tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
# config = AutoConfig.from_pretrained("microsoft/MiniLM-L12-H384-uncased")#AutoConfig.from_pretrained("distilbert-base-uncased")

class MiniLMQA(PreTrainedModel):
    def __init__(self,config: PretrainedConfig):
        # super(DistillBERTQA, config).__init__()
        super().__init__(config)
        self.MiniLM = AutoModel.from_pretrained("microsoft/MiniLM-L12-H384-uncased") #DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.qa_outputs = torch.nn.Linear(384, 2)
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, input_ids=None, attention_mask=None,start_positions=None,end_positions=None,return_dict=None):
        minilm_output = self.MiniLM(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = minilm_output[0]
        hidden_states = self.dropout(hidden_states)
        logits = self.qa_outputs(hidden_states)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous()  # (bs, max_query_len)
        end_logits = end_logits.squeeze(-1).contiguous()  # (bs, max_query_len)
        total_loss = None
        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            # sometimes the start/end positions are outside our model inputs, we ignore these terms
            ignored_index = start_logits.size(1)
            start_positions = start_positions.clamp(0, ignored_index)
            end_positions = end_positions.clamp(0, ignored_index)

            loss_fct = torch.nn.CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            total_loss = (start_loss + end_loss) / 2

        if not return_dict:
            output = (start_logits, end_logits) + minilm_output[1:]
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=minilm_output.hidden_states,
            attentions=minilm_output.attentions,
        )



In [4]:
tokenizer = AutoTokenizer.from_pretrained('/home/vp.shivasan/interiit/task2/training_dir/task2_MiniLM-mod_50epochs_2e-5_FINAL_ES/checkpoint-5076')
config = AutoConfig.from_pretrained('/home/vp.shivasan/interiit/task2/training_dir/task2_MiniLM-mod_50epochs_2e-5_FINAL_ES/checkpoint-5076')
model = MiniLMQA(config)


In [5]:
state_dict = torch.load('/home/vp.shivasan/interiit/task2/training_dir/task2_MiniLM-mod_50epochs_2e-5_FINAL_ES/checkpoint-5076/pytorch_model.bin')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [6]:
# text = r"""
# 🤗 The NES uses a custom-made Picture Processing Unit (PPU) developed by Ricoh. All variations of the PPU feature 2 kB of video RAM, 256 bytes of on-die ""object attribute memory"" (OAM) to store the positions, colors, and tile indices of up to 64 sprites on the screen, and 28 bytes of on-die palette RAM to allow selection of background and sprite colors. The console's 2 kB of onboard RAM may be used for tile maps and attributes on the NES board and 8 kB of tile pattern ROM or RAM may be included on a cartridge. The system has an available color palette of 48 colors and 6 grays. Up to 25 simultaneous colors may be used without writing new values mid-frame: a background color, four sets of three tile colors and four sets of three sprite colors. The NES palette is based on NTSC rather than RGB values. A total of 64 sprites may be displayed onscreen at a given time without reloading sprites mid-screen. The standard display resolution of the NES is 256 horizontal pixels by 240 vertical pixels.
# """
# question=  "Who developed Nintendo's PPU?"

def get_answer(question,context):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    model_inputs = inputs
    del model_inputs['token_type_ids']
    output = model(**model_inputs)

    answer_start_scores, answer_end_scores,_ = model(**inputs)

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    return answer

# print(f"Question: {question}")
# print(f"Answer: {answer}\n")

In [7]:
my_df = pd.read_csv('/home/vp.shivasan/interiit/data/Task2dataSet_test.csv')
F1s = []
n_big_para = 0
for i in tqdm(range(len(my_df))):
    question = my_df['question'][i]
    context = my_df['context'][i]
    correct_answer = ast.literal_eval(my_df['answers'][i])['text'][0]
    try:
        predicted_answer = get_answer(question=question,context=context)
    except:
        print(i,question)
        n_big_para+=1
        continue
    # predicted_answer = question_answerer(question=question, context=context)['answer']
    F1s.append(compute_f1(correct_answer,predicted_answer))

print("Average F1 score",np.mean(F1s))
print("Number of huge paras: ",n_big_para)

  2%|▏         | 125/5013 [00:08<05:00, 16.29it/s]

122 Which journalist considered Spectre the worst James Bond movie in three decades?


  9%|▉         | 443/5013 [00:27<04:14, 17.95it/s]

439 Which writer for the San Francisco Chronicle awarded Spectre with a perfect score?


 12%|█▏        | 604/5013 [00:37<03:55, 18.73it/s]

...

4035 To whom was Eclogue 4 addressed to?


 86%|████████▋ | 4336/5013 [04:22<00:37, 18.29it/s]

4332 Who thought Burke was prophetic about the French revolution's consequences?
4336 During daytime how high can the temperatures reach?


100%|██████████| 5013/5013 [05:01<00:00, 16.63it/s]

Average F1 score 0.7506395323328671
Number of huge paras:  10


In [27]:
output[2].shape

torch.Size([1, 384])

# Load from automodel

In [9]:
tokenizer = AutoTokenizer.from_pretrained('/home/vp.shivasan/interiit/task2/training_dir/task2_MiniLM-mod_50epochs_2e-5_FINAL_ES/checkpoint-5076')
model = AutoModel.from_pretrained('/home/vp.shivasan/interiit/task2/training_dir/task2_MiniLM-mod_50epochs_2e-5_FINAL_ES/checkpoint-5076',local_files_only = True,return_dict = True)
config = AutoConfig.from_pretrained('/home/vp.shivasan/interiit/task2/training_dir/task2_MiniLM-mod_50epochs_2e-5_FINAL_ES/checkpoint-5076')

Some weights of the model checkpoint at /home/vp.shivasan/interiit/task2/training_dir/task2_MiniLM-mod_50epochs_2e-5_FINAL_ES/checkpoint-5076 were not used when initializing BertModel: ['MiniLM.encoder.layer.2.output.dense.bias', 'MiniLM.encoder.layer.9.intermediate.dense.bias', 'MiniLM.encoder.layer.8.attention.self.value.weight', 'MiniLM.encoder.layer.7.intermediate.dense.bias', 'MiniLM.encoder.layer.1.attention.self.query.weight', 'MiniLM.encoder.layer.0.attention.output.dense.bias', 'MiniLM.encoder.layer.9.attention.self.key.weight', 'MiniLM.encoder.layer.10.attention.output.dense.bias', 'MiniLM.encoder.layer.7.output.LayerNorm.bias', 'MiniLM.encoder.layer.7.output.dense.bias', 'MiniLM.encoder.layer.0.attention.self.value.bias', 'MiniLM.encoder.layer.7.attention.self.value.weight', 'MiniLM.encoder.layer.7.attention.self.query.weight', 'MiniLM.encoder.layer.2.attention.output.dense.bias', 'MiniLM.encoder.layer.11.intermediate.dense.weight', 'MiniLM.encoder.layer.2.attention.self.val

In [10]:
inputs = tokenizer.encode_plus(question, para, add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"].tolist()[0]
text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
output = model(**inputs)

In [11]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-2.8435, -0.1767, -0.0284,  ..., -0.9489, -0.6321,  0.9973],
         [-1.4692,  0.3385, -1.4302,  ..., -1.0068, -2.2559,  0.5336],
         [-2.1760,  0.7128, -1.5751,  ..., -0.0490, -1.0524,  1.2547],
         ...,
         [ 1.9317, -0.0853,  0.9058,  ..., -0.8918, -0.3346,  0.1205],
         [ 1.4556,  1.5898,  2.5926,  ..., -0.3540, -0.2982,  1.9612],
         [ 1.2073,  0.6247,  0.2067,  ..., -1.3080, -1.0914,  1.8486]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-3.7681e-01, -1.0134e-01,  6.8426e-01,  5.6293e-01, -2.6529e-01,
          3.2416e-01, -9.0883e-02,  5.4870e-01,  1.5242e-01, -4.8494e-01,
         -2.1586e-01,  1.8552e-01,  7.1669e-02, -7.1331e-03,  2.7147e-01,
         -3.2243e-01,  1.2285e-01, -3.1309e-02,  4.9929e-01,  1.8394e-01,
         -2.9855e-01,  2.1637e-02,  4.3720e-01,  1.7160e-02,  1.3632e-01,
          2.9443e-01, -5.1914e-02,  3.5638e-01,  3.3920e-01,  7.115